In [2]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 30.0 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score

2024-08-02 11:49:24.391530: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-02 11:49:24.399690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 11:49:24.408959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 11:49:24.411715: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 11:49:24.418703: I tensorflow/core/platform/cpu_feature_guar

In [23]:
data = pd.read_csv('updated_data.csv')
text_data = data['tweet_text'].values
numerical_data = data[['retweet_count', 'followers_count', 'verified',
                           'time_delay (min)', 'neg_sent', 'pos_sent', 
                       'neu_sent', 'comp_sent', 'difference', 'ratio']].values
y = data['label']
data.head()

,tweet_text,retweet_count,favorite_count,followers_count,friends_count,verified,label,time_delay (min),neg_sent,pos_sent,neu_sent,comp_sent,difference,weight,ratio
0,"Reports of ""moving body"" amidst #Germanwings w...",38,15,337960,6384,1,1,1.2833,0.000,0.00,1.000,0.0000,0.0,0.0,1.000000e+00
1,BREAKING:148passengers were on board #GermanWi...,43,15,52815,293,0,1,1.5000,0.000,0.14,0.860,0.3818,1.0,1.0,1.000000e+10
2,BREAKING: #Germanwings crash victims include 7...,31,5,893549,2312,1,1,4.1333,0.278,0.00,0.722,-0.6124,-0.2,-0.2,6.666667e-01
3,BREAKING: 148 feared dead in crashed #Germanwi...,167,32,418641,1859,1,1,3.5167,0.517,0.00,0.483,-0.8176,0.0,0.0,1.000000e+00
4,Terrible news... Airbus A320 from Barcelona to...,26,1,11062,233,0,1,3.3667,0.171,0.00,0.829,-0.4767,0.0,0.0,1.000000e+00


In [3]:
# we added stratifying sampling
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(text_data, numerical_data, y, 
                                                                                       test_size=0.2, 
                                                                                       random_state=42, stratify=y)
print(f'Train text: {len(X_train_text)}'), print(f'Train number: {len(X_train_num)}')
print(f'Test text: {len(X_test_text)}'), print(f'Test number:{len(X_test_num)}')
print(f'Y Training: {len(y_train)}'), print(f'Y Testing:{len(y_test)}')

Train text: 4641
Train number: 4641
Test text: 1161
Test number:1161
Y Training: 4641
Y Testing:1161


(None, None)

## Preprocess Data

In [7]:
import os
os.environ["CUDA_VISIBLE_DIVICES"]='-1'

In [8]:
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [9]:
def elmo_embeddings(text_list):
    embeddings = elmo.signatures['default'](tf.constant(text_list))['elmo']
    return embeddings

In [10]:
def process_in_batches(texts, batch_size=32):
    elmo_embed_np = []
    total_batches = (len(texts) + batch_size - 1) // batch_size  # Calculate total number of batches
    with tqdm(total=total_batches, desc='Processing Batches') as pbar:
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = elmo_embeddings(batch_texts)
            # Average the embeddings for each text
            batch_embeddings_np = [np.mean(embed.numpy(), axis=0) for embed in batch_embeddings]
            for embed in batch_embeddings_np:
                if embed.shape != (1024,):
                    print(f"Unexpected shape found: {embed.shape}")
            elmo_embed_np.extend(batch_embeddings_np)
            pbar.update(1)  # Update progress bar
    return np.array(elmo_embed_np)

In [11]:
train_elmo_embed_np = process_in_batches(X_train_text, batch_size=16)  # Adjust batch size as needed
test_elmo_embed_np = process_in_batches(X_test_text, batch_size=16)  # Adjust batch size as needed
test_elmo_embed_np.shape

Processing Batches: 100%|███████████████████████| 73/73 [01:08<00:00,  1.07it/s]


(1161, 1024)

## Numerical features

In [12]:
#using custom range
class CustomMinMaxScaler:
    def __init__(self, feature_range=(-2, 2)):
        self.feature_range = feature_range
        self.scaler = MinMaxScaler(feature_range=(0, 1))
    
    def fit(self, X):
        # Fit the scaler to the data
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        # Transform the data to [0, 1] range
        X_normalized = self.scaler.transform(X)
        # Scale to the desired range [-2, 2]
        a, b = self.feature_range
        X_scaled = a + (X_normalized * (b - a))
        return X_scaled
    
    def fit_transform(self, X):
        # Fit and transform the data
        return self.fit(X).transform(X)

In [13]:
num_scaler = CustomMinMaxScaler(feature_range=(-2, 2))
train_norm_num_features = num_scaler.fit_transform(X_train_num)
test_norm_num_features = num_scaler.fit_transform(X_test_num)
print(f'Normalized test shape: {test_norm_num_features.shape}')

Normalized test shape: (1161, 10)


## Concatenate Features

In [16]:
X_train = np.hstack((train_elmo_embed_np, train_norm_num_features))
X_test = np.hstack((test_elmo_embed_np, test_norm_num_features))
X_test.shape

(1161, 1034)

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
xg_model = XGBClassifier(random_state=42)

param_grid = {
    'max_depth': [6, 10, 20, None]
}
grid_search = GridSearchCV(xg_model, param_grid, cv=10, verbose=0, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'max_depth': [6, 10, 20, None]}, scoring='accuracy')

In [18]:
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

Best parameters: {'max_depth': 6}


In [19]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
xg_model = XGBClassifier(max_depth=6, random_state=42) # Initialize a decision tree regressor
scoring = ['accuracy', 'recall',  'precision','f1_macro', 'f1_weighted' ]
scores = cross_validate(xg_model, X_train, y_train, scoring=scoring, cv=10)
xg_cv_scores = pd.DataFrame(scores)
display(xg_cv_scores)

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1_macro,test_f1_weighted
0,10.352486,0.007897,0.821505,0.683544,0.765957,0.795435,0.818836
1,10.249201,0.008758,0.862069,0.764331,0.816327,0.843455,0.860906
2,9.578932,0.007580,0.836207,0.732484,0.771812,0.814724,0.835119
3,10.533098,0.007418,0.851293,0.694268,0.838462,0.825969,0.847430
4,9.258564,0.008337,0.814655,0.689873,0.746575,0.789642,0.812779
5,9.172580,0.007651,0.825431,0.670886,0.785185,0.797995,0.821741
6,9.218790,0.007374,0.849138,0.683544,0.843750,0.823105,0.844750
7,8.560086,0.007292,0.831897,0.708861,0.777778,0.808561,0.829880
8,8.619913,0.007286,0.808190,0.664557,0.744681,0.780423,0.805329
9,8.497644,0.007662,0.836207,0.708861,0.788732,0.812824,0.833926


In [20]:
avg_f1_macro_cv_score = xg_cv_scores['test_f1_macro'].mean()
avg_f1_weighted_cv_score = xg_cv_scores['test_f1_weighted'].mean()
avg_precision_cv_score = xg_cv_scores['test_precision'].mean()
avg_recall_cv_score = xg_cv_scores['test_recall'].mean()
avg_acc_cv_score = xg_cv_scores['test_accuracy'].mean()

acc_perc = round((avg_acc_cv_score * 100), 2)
recall_perc = round((avg_recall_cv_score * 100), 2)
precision_perc = round((avg_precision_cv_score * 100), 2)
weighted_perc = round((avg_f1_weighted_cv_score * 100), 2)
macro_perc = round((avg_f1_macro_cv_score * 100), 2)

print(f'Precision: {avg_precision_cv_score}', f'| Prec %: {precision_perc}')
print(f'Recall: {avg_recall_cv_score}', f' | Recall %: {recall_perc}')
print(f'Accuracy: {avg_acc_cv_score}', f' | Accuracy %: {acc_perc}')
print(f'Macro F1: {avg_f1_macro_cv_score}', f' | Macro F1 %: {macro_perc}')
print(f'Weighted F1: {avg_f1_weighted_cv_score}', f' | Weighted F1 %: {weighted_perc}')

Precision: 0.787925914727795 | Prec %: 78.79
Recall: 0.7001209384826251  | Recall %: 70.01
Accuracy: 0.8336591583240637  | Accuracy %: 83.37
Macro F1: 0.8092133289636537  | Macro F1 %: 80.92
Weighted F1: 0.8310695167868953  | Weighted F1 %: 83.11


In [21]:
xg_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [26]:
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score,f1_score
y_test_pred = xg_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)

test_acc_perc = round((test_accuracy * 100), 2)
test_recall_perc = round((test_recall * 100), 2)
test_precision_perc = round((test_precision * 100), 2)

print(f'Test Precision: {test_precision}', f" | Test Set Accuracy %: {test_acc_perc}")
print(f'Training Set Recall: {test_recall}', f' | Test Set Recall %: {test_recall_perc}')
print(f"Training Set Accuracy: {test_accuracy}", f' | Test Set Precision %: {test_precision_perc}')

from sklearn.metrics import f1_score
macrof1_score = f1_score(y_test, y_test_pred, average='macro')
macro_perc = round((macrof1_score * 100), 2)
print(f'Macro F1: {macrof1_score}',f' | Test Macro F1 %: {macro_perc}')

wf1_score = f1_score(y_test, y_test_pred, average='weighted')
weighted_perc = round((wf1_score * 100), 2)
print(f'Weighted F1: {wf1_score}',f' | Test Weighted F1 %: {weighted_perc}')

Test Precision: 0.7900874635568513  | Test Set Accuracy %: 83.12
Training Set Recall: 0.6860759493670886  | Test Set Recall %: 68.61
Training Set Accuracy: 0.8311800172265289  | Test Set Precision %: 79.01
Macro F1: 0.8053399852180341  | Test Macro F1 %: 80.53
Weighted F1: 0.8280034648283214  | Test Weighted F1 %: 82.8


# Model Optimization with All Features

In [29]:
def xgboost_cv(loss, subsample,learning_rate, n_estimators, min_sample_split, min_samples_leaf, min_weight_fraction_leaf, 
               max_depth):
    loss_mapping = {0: 'log_loss', 1:'exponential'}
    loss_used = loss_mapping[int(round(loss))]
    n_estimators = int(round(n_estimators))
    max_depth = int(round(max_depth))
    
    
    clf = GradientBoostingClassifier(loss = loss_used,
                                     subsample =subsample,
                                     learning_rate=learning_rate,
                                     n_estimators=n_estimators, 
                                     max_depth=max_depth, 
                                     min_samples_leaf = min_samples_leaf,
                                     min_weight_fraction_leaf = min_weight_fraction_leaf,
                                     random_state=123)
    
    kfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=123)
    score = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=kfold)

    score = np.nan_to_num(score)
    score = score.mean()
    return score

In [25]:
gb_params={
    'loss':(0,1),
    'subsample':(0.0, 1.0),
    'learning_rate':(0.001, 0.1),
    'n_estimators':(50,150),
    'min_sample_split': (0.0, 1.0),
    'min_samples_leaf': (0.0, 1.0),
    'min_weight_fraction_leaf': (0.0, 0.5),
    'max_depth': (3, 50),
    
}

In [30]:
xg_bo_optimizer = BayesianOptimization(xgboost_cv, gb_params, random_state = 111)
xg_bo_optimizer.maximize(n_iter=100)

|   iter    |  target   | learni... |   loss    | max_depth | min_sa... | min_sa... | min_we... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
| 1         | 0.6602    | 0.0616    | 0.1691    | 23.49     | 0.7693    | 0.2953    | 0.07458   | 52.25     | 0.4202    |
| 2         | 0.7712    | 0.02463   | 0.3377    | 49.56     | 0.2377    | 0.08119   | 0.3348    | 112.1     | 0.2743    |
| 3         | 0.6602    | 0.04716   | 0.1184    | 6.476     | 0.9008    | 0.794     | 0.4203    | 131.5     | 0.991     |
| 4         | 0.6602    | 0.05815   | 0.8138    | 22.8      | 0.02745   | 0.4541    | 0.05266   | 131.7     | 0.6977    |
| 5         | 0.6602    | 0.05696   | 0.2742    | 49.93     | 0.138     | 0.6154    | 0.2425    | 90.5      | 0.7278    |
| 6         | 0.6602    | 0.0558    | 0.9131    | 48.92     | 0.4764    | 0.7137    | 0.0353    | 116.4     | 0.8733    |
| 7         | 0.7873    

In [32]:
# # Print the best result
print(xg_bo_optimizer.max)

{'target': 0.8146976262944954, 'params': {'learning_rate': 0.05840467107355432, 'loss': 0.4156407003860343, 'max_depth': 40.710230289568486, 'min_sample_split': 0.4168523275656557, 'min_samples_leaf': 0.05829396945676946, 'min_weight_fraction_leaf': 0.06573837227864238, 'n_estimators': 107.99728502594091, 'subsample': 0.8456603673383449}}


In [33]:
# Extract the results
results = xg_bo_optimizer.res
# Create a DataFrame from the results
results_df = pd.DataFrame(results)
# Optionally, save the DataFrame to a CSV file
results_df.to_csv('xg_bayesian_optimization_results.csv', index=False)

In [34]:
import json
# Extract the best result
best_result = xg_bo_optimizer.max

# Save the best result to a JSON file
with open('xg_best_result.json', 'w') as f:
    json.dump(best_result, f)

# Model Testing with Optimal Parameters

In [39]:
def xgboost_model(loss, subsample,learning_rate, n_estimators, min_sample_split, min_samples_leaf, min_weight_fraction_leaf, 
               max_depth):
    loss_mapping = {0: 'log_loss', 1:'exponential'}
    loss_used = loss_mapping[int(round(loss))]
    n_estimators = int(round(n_estimators))
    max_depth = int(round(max_depth))
    
    
    model = GradientBoostingClassifier(loss = loss_used,
                                     subsample =subsample,
                                     learning_rate=learning_rate,
                                     n_estimators=n_estimators, 
                                     max_depth=max_depth, 
                                     min_samples_leaf = min_samples_leaf,
                                     min_weight_fraction_leaf = min_weight_fraction_leaf,
                                     random_state=123)
    
    return model

In [51]:
loss=0.4156407003860343
subsample = 0.8456603673383449
learning_rate= 0.05840467107355432
n_estimators = 107.99728502594091
max_depth= 40.710230289568486
min_samples_leaf = 0.05829396945676946
min_sample_split = 0.4168523275656557
min_weight_fraction_leaf = 0.06573837227864238

model = xgboost_model(loss, subsample,learning_rate, n_estimators, min_sample_split, min_samples_leaf, min_weight_fraction_leaf, max_depth)

model.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.05840467107355432, max_depth=41,
                           min_samples_leaf=0.05829396945676946,
                           min_weight_fraction_leaf=0.06573837227864238,
                           n_estimators=108, random_state=123,
                           subsample=0.8456603673383449)

### Train

In [57]:
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Set Accuracy: {train_accuracy}")

train_recall = recall_score(y_train, y_train_pred)
print(f'Training Set Recall: {train_recall}')

train_precision = precision_score(y_train, y_train_pred)
print(f'Training Set Precision: {train_precision}')

Training Set Accuracy: 0.8995906054729584
Training Set Recall: 0.8084971464806595
Training Set Precision: 0.8860319666435025


In [59]:
train_acc_perc = round((train_accuracy * 100), 2)
train_recall_perc = round((train_recall * 100), 2)
train_precision_perc = round((train_precision * 100), 2)

print(f"Training Set Accuracy %: {train_acc_perc}")
print(f'Training Set Recall %: {train_recall_perc}')
print(f'Training Set Precision %: {train_precision_perc}')

Training Set Accuracy %: 89.96
Training Set Recall %: 80.85
Training Set Precision %: 88.6


In [61]:
from sklearn.metrics import f1_score
train_macrof1_score = f1_score(y_train, y_train_pred, average='macro')
train_macro_perc = round((train_macrof1_score * 100), 2)
print(train_macrof1_score)

print(f'Macro F1 %: {train_macro_perc}')

train_wf1_score = f1_score(y_train, y_train_pred, average='weighted')
train_weighted_perc = round((train_wf1_score * 100), 2)
print(train_wf1_score)

print(f'Weighted F1 %: {train_weighted_perc}')

0.8855605511958352
Macro F1 %: 88.56
0.8983991300943475
Weighted F1 %: 89.84


### Test

In [52]:
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Training Set Accuracy: {test_accuracy}")

test_recall = recall_score(y_test, y_test_pred)
print(f'Training Set Recall: {test_recall}')

test_precision = precision_score(y_test, y_test_pred)
print(f'Training Set Precision: {test_precision}')

Training Set Accuracy: 0.8105081826012058
Training Set Recall: 0.6329113924050633
Training Set Precision: 0.7692307692307693


In [60]:
test_acc_perc = round((test_accuracy * 100), 2)
test_recall_perc = round((test_recall * 100), 2)
test_precision_perc = round((test_precision * 100), 2)

print(f"Test Set Accuracy %: {test_acc_perc}")
print(f'Test Set Recall %: {test_recall_perc}')
print(f'Test Set Precision %: {test_precision_perc}')

Test Set Accuracy %: 81.05
Test Set Recall %: 63.29
Test Set Precision %: 76.92


In [62]:
from sklearn.metrics import f1_score
test_macrof1_score = f1_score(y_test, y_test_pred, average='macro')
test_macro_perc = round((test_macrof1_score * 100), 2)
print(test_macrof1_score)

print(f'Macro F1 %: {test_macro_perc}')

test_wf1_score = f1_score(y_test, y_test_pred, average='weighted')
test_weighted_perc = round((test_wf1_score * 100), 2)
print(test_wf1_score)

print(f'weighted F1 %: {test_weighted_perc}')

0.7785580524344569
Macro F1 %: 77.86
0.8054367333683886
weighted F1 %: 80.54


# Model with baseline parameters

In [53]:
base_model = GradientBoostingClassifier(loss = 'log_loss',
                                     subsample =1.0,
                                     learning_rate=0.1,
                                     n_estimators=100, 
                                     max_depth=3, 
                                     min_samples_leaf = 1,
                                     min_weight_fraction_leaf = 0.0,
                                     random_state=123)
    
base_model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=123)

### Training

In [58]:
base_train_pred = base_model.predict(X_train)
base_train_accuracy = accuracy_score(y_train, base_train_pred)
print(f"Training Set Accuracy: {base_train_accuracy}")

base_train_recall = recall_score(y_train, base_train_pred)
print(f'Training Set Recall: {base_train_recall}')

base_train_precision = precision_score(y_train, base_train_pred)
print(f'Training Set Precision: {base_train_precision}')

Training Set Accuracy: 0.9105796164619694
Training Set Recall: 0.8192771084337349
Training Set Precision: 0.9085794655414908


In [63]:
from sklearn.metrics import f1_score
base_train_acc_perc = round((base_train_accuracy * 100), 2)
base_train_recall_perc = round((base_train_recall * 100), 2)
base_train_precision_perc = round((base_train_precision * 100), 2)

print(f"Training Set Accuracy %: {base_train_acc_perc}")
print(f'Training Set Recall %: {base_train_recall_perc}')
print(f'Training Set Precision %: {base_train_precision_perc}')

base_train_macrof1_score = f1_score(y_train, base_train_pred, average='macro')
base_train_macro_perc = round((base_train_macrof1_score * 100), 2)
print(base_train_macrof1_score)

print(f'Macro F1 %: {base_train_macro_perc}')

base_train_wf1_score = f1_score(y_train, base_train_pred, average='weighted')
base_train_weighted_perc = round((base_train_wf1_score * 100), 2)
print(base_train_wf1_score)

print(f'Weighted F1 %: {base_train_weighted_perc}')

Training Set Accuracy %: 91.06
Training Set Recall %: 81.93
Training Set Precision %: 90.86
0.8977846453884544
Macro F1 %: 89.78
0.909371808595712
Weighted F1 %: 90.94


### Testing

In [56]:
base_y_test_pred = base_model.predict(X_test)
base_test_accuracy = accuracy_score(y_test, base_y_test_pred)
print(f"Training Set Accuracy: {base_test_accuracy}")

base_test_recall = recall_score(y_test, base_y_test_pred)
print(f'Training Set Recall: {base_test_recall}')

base_test_precision = precision_score(y_test, base_y_test_pred)
print(f'Training Set Precision: {base_test_precision}')

Training Set Accuracy: 0.8096468561584841
Training Set Recall: 0.6506329113924051
Training Set Precision: 0.7558823529411764


In [64]:
base_test_acc_perc = round((base_test_accuracy * 100), 2)
base_test_recall_perc = round((base_test_recall * 100), 2)
base_test_precision_perc = round((base_test_precision * 100), 2)

print(f"Test Set Accuracy %: {base_test_acc_perc}")
print(f'Test Set Recall %: {base_test_recall_perc}')
print(f'Test Set Precision %: {base_test_precision_perc}')

base_test_macrof1_score = f1_score(y_test, base_y_test_pred, average='macro')
base_test_macro_perc = round((base_test_macrof1_score * 100), 2)
print(base_test_macro_perc)

print(f'Macro F1 %: {base_test_macro_perc}')

base_test_wf1_score = f1_score(y_test, base_y_test_pred, average='weighted')
base_test_weighted_perc = round((base_test_wf1_score * 100), 2)
print(base_test_weighted_perc)

print(f'Weighted F1 %: {base_test_weighted_perc}')

Test Set Accuracy %: 80.96
Test Set Recall %: 65.06
Test Set Precision %: 75.59
78.0
Macro F1 %: 78.0
80.58
Weighted F1 %: 80.58
